In [39]:
import numpy as np
import tensorflow as tf
from skimage.io import imread_collection
from sklearn.model_selection import train_test_split
import keras
from matplotlib import pyplot as plt

# Fix the random seed
tf.random.set_seed(0)
np.random.seed(0)

In [40]:
# Load data to tf.data.Dataset objects
# Train data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "GTSRB_subset_2",
    validation_split=0.2,
    subset="training",
    seed=0,
    image_size=(64, 64),
    batch_size=32,
    shuffle = True)

# Test data
val_ds =tf.keras.preprocessing.image_dataset_from_directory(
    "GTSRB_subset_2",
    validation_split=0.2,
    subset="validation",
    seed=0,
    image_size=(64, 64),
    batch_size=32,
    shuffle = True)

Found 660 files belonging to 2 classes.
Using 528 files for training.
Found 660 files belonging to 2 classes.
Using 132 files for validation.


In [42]:
batch_size = 32
epochs = 10

with tf.device('/gpu:0'):
    model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(64, 64, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(momentum = 0.9),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(momentum = 0.9),
    tf.keras.layers.Dense(1, activation = "sigmoid")
    ])
    
    # Compile model
    model.compile(optimizer=tf.optimizers.SGD(0.001), loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    #history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, class_weight = class_weight_dict)
    history = model.fit(train_ds, batch_size=batch_size, epochs=epochs, verbose=1)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_9 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 flatten_9 (Flatten)         (None, 12288)             0         
                                                                 
 dense_27 (Dense)            (None, 100)               1228900   
                                                                 
 batch_normalization_8 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_28 (Dense)            (None, 100)               10100     
                                                                 
 batch_normalization_9 (Batc  (None, 100)              400       
 hNormalization)                                      

In [43]:
# Evaluate test data
predict = model.evaluate(val_ds)
print(f"Test loss: {predict[0]} - Test accuracy: {predict[1]}")

5/5 [==============================] - 0s 3ms/step - loss: 0.0502 - accuracy: 1.0000
Test loss: 0.05020425468683243 - Test accuracy: 1.0
